## Projeto Final - Primeira Entrega

In [255]:
import pandas as pd
import requests 
from datetime import datetime
from plyer import notification

In [256]:
#Função para exibir as notificações de erro no carregamento da API
def alerta(api_nome, status_code):
    notification.notify(
        title="Erro na API",
        message=f"Falha no carregamento da API {api_nome}\nErro: {status_code}\n{datetime.now()}",
        app_name="alerta",
        timeout=10
    )

In [257]:
#Criando dicionário com as URLs das APIs
apis = {
    "temperaturas": "https://global-warming.org/api/temperature-api",
    "gelo": "https://global-warming.org/api/arctic-api",
    "oceanos": "https://global-warming.org/api/ocean-warming-api"
}

def carregar_api(url, nome_api):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        data
        
        # Transformar os dados em um dataframe
        try:
            df = pd.DataFrame(data)
            print(f"Primeiras linhas do DataFrame para a API {nome_api}:\n{df.head()}")
            return df
        except ValueError:
            alerta(nome_api, "Dados retornados não podem ser convertidos em um dataframe")
            return None
    else:
        alerta(nome_api, f"Erro ao acessar API. Código HTTP: {response.status_code}")
        return None

#Carregando os dados
for nome_api, url in apis.items():
    carregar_api(url, nome_api)


Primeiras linhas do DataFrame para a API temperaturas:
  error                                             result
0  None  {'time': '1880.04', 'station': '-0.31', 'land'...
1  None  {'time': '1880.13', 'station': '-0.51', 'land'...
2  None  {'time': '1880.21', 'station': '-0.34', 'land'...
3  None  {'time': '1880.29', 'station': '-0.55', 'land'...
4  None  {'time': '1880.38', 'station': '-0.24', 'land'...
Primeiras linhas do DataFrame para a API gelo:
            error                                         arcticData
description  None  {'title': 'Global Sea Ice Extent (1979-2024)',...
data         None  {'197901': {'value': 20.81, 'anom': 1.79, 'mon...
Primeiras linhas do DataFrame para a API oceanos:
     error result
1851  None  -0.05
1852  None   0.01
1853  None   0.06
1854  None   0.00
1855  None   0.00


In [258]:
# checar o type dos itens
data_temp.info()
data_ice.info()
data_ocean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1738 entries, 0 to 1737
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   error   0 non-null      object
 1   result  1738 non-null   object
dtypes: object(2)
memory usage: 27.3+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, description to data
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   error       0 non-null      object
 1   arcticData  2 non-null      object
dtypes: object(2)
memory usage: 156.0+ bytes
<class 'pandas.core.frame.DataFrame'>
Index: 174 entries, 1851 to 2024
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   error   0 non-null      object
 1   result  174 non-null    object
dtypes: object(2)
memory usage: 4.1+ KB


## Transformando as bases em colunas no dataframe

### tamanho gelo - milhões de m²

In [268]:
# Extrair o dicionário dentro de 'arcticData'
if data_ice is not None and "arcticData" in data_ice:
    arctic_data = data_ice["arcticData"]["data"]  
    table_ice = pd.DataFrame.from_dict(arctic_data, orient="index").reset_index()
    table_ice.columns = ["date", "value", "anom", "month"] 
    print("Tabela de Gelo:\n", table_ice)
    print(table_ice.iloc[105:120])
 
else:
    print("Erro na exibição dos dados vindos da API de Gelo")

Tabela de Gelo:
        date  value  anom  month
0    197901  20.81  1.79  19.02
1    197902  19.32  1.37  17.95
2    197903  20.34  1.23  19.11
3    197904  22.94  1.67  21.27
4    197905  24.69  1.47  23.22
..      ...    ...   ...    ...
545  202406  22.66 -2.10  24.76
546  202407  22.00 -2.82  24.82
547  202408  21.55 -2.72  24.27
548  202409  21.44 -2.76  24.20
549  202410  22.49 -3.23  25.72

[550 rows x 4 columns]
       date    value     anom  month
105  198710    26.72     1.00  25.72
106  198711    26.80     0.79  26.01
107  198712 -9999.00 -9999.00  22.81
108  198801 -9999.00 -9999.00  19.02
109  198802    18.47     0.52  17.95
110  198803    19.93     0.82  19.11
111  198804    21.41     0.14  21.27
112  198805    23.16    -0.06  23.22
113  198806    25.03     0.27  24.76
114  198807    25.56     0.74  24.82
115  198808    25.45     1.18  24.27
116  198809    25.68     1.48  24.20
117  198810    27.49     1.77  25.72
118  198811    27.48     1.47  26.01
119  198812    24.69

In [274]:
# Refazendo a transformação e removendo as linhas 107 e 108
if data_ice is not None and "arcticData" in data_ice:
    # Extrair e processar os dados
    arctic_data = data_ice["arcticData"]["data"]
    table_ice = pd.DataFrame.from_dict(arctic_data, orient="index").reset_index()
    table_ice.columns = ["date", "value", "anom", "month"]

    # Adicionar coluna 'year' e remover colunas desnecessárias
    table_ice['year'] = table_ice['date'].str[:4].astype(int)
    table_ice = table_ice.drop(columns=['anom', 'date'])

    # Remover as linhas 107 e 108 - valores outliers
    table_ice.drop(index=[107, 108], inplace=True)

    # Calcular a média anual de 'value'
    table_ice = (
        table_ice.groupby('year', as_index=False)['value']
        .mean()
        .rename(columns={'value': 'media anual - extensão do gelo'})
    )

    # Exibir o resultado
    print("Média Anual da Extensão do Gelo:\n", table_ice)

else:
    print("Erro na exibição ou transformação dos dados vindos da API de Gelo")

Média Anual da Extensão do Gelo:
     year  media anual - extensão do gelo
0   1979                       24.005833
1   1980                       23.554167
2   1981                       23.533333
3   1982                       24.062500
4   1983                       23.697500
5   1984                       23.348333
6   1985                       23.585000
7   1986                       23.268333
8   1987                       23.663636
9   1988                       24.031818
10  1989                       23.367500
11  1990                       23.082500
12  1991                       23.268333
13  1992                       23.495833
14  1993                       23.316667
15  1994                       23.760000
16  1995                       23.186667
17  1996                       23.591667
18  1997                       23.032500
19  1998                       23.470833
20  1999                       23.425833
21  2000                       23.243333
22  2001               

### anomalia na temperatura de oceano

In [271]:
# Transformar a base de Oceanos em DF e remover a coluna error e adicionar year
if data_ocean is not None:
    table_ocean = pd.DataFrame(data_ocean)
    table_ocean = table_ocean.drop(columns=['error']).reset_index()
    table_ocean.columns = ['year', 'result']
    table_ocean['year'] = table_ocean['year'].astype(int)
    table_ocean['result'] = table_ocean['result'].astype(float)
    print("Global Ocean Temperature Anomalies:\n", table_ocean)
else:
    print("Erro transformação API de Oceano")

Global Ocean Temperature Anomalies:
      year  result
0    1851   -0.05
1    1852    0.01
2    1853    0.06
3    1854    0.00
4    1855    0.00
..    ...     ...
169  2020    0.75
170  2021    0.70
171  2022    0.64
172  2023    0.67
173  2024    0.94

[174 rows x 2 columns]


### anomalia temperatura global

In [272]:
# Transformar a base de Temperaturas com tolist()
if data_temp is not None and not data_temp.empty and "result" in data_temp:
    table_temp = pd.DataFrame(data_temp["result"].tolist()) 
    table_temp['year'] = table_temp['time'].astype(str).str[:4].astype(int)
    convert = ['station', 'land']
    table_temp[convert] = table_temp[convert].astype(float)
    
    # Agrupar por ano e calcular a média para 'station' e 'land'
    table_temp = (
        table_temp.groupby('year')[convert].mean().reset_index()
        .rename(columns={'station': 'media_station', 'land': 'media_land'})
    )
    print("Média anual de 'station' e 'land':\n", table_temp)
else:
    print("Erro transformação API Temperaturas")


Média anual de 'station' e 'land':
      year  media_station  media_land
0    1880      -0.356667   -0.180000
1    1881      -0.289167   -0.092500
2    1882      -0.393333   -0.113333
3    1883      -0.377500   -0.175833
4    1884      -0.508333   -0.285000
..    ...            ...         ...
140  2020       1.315833    1.007500
141  2021       1.143333    0.849167
142  2022       1.163333    0.894167
143  2023       1.441667    1.173333
144  2024       1.542000    1.285000

[145 rows x 3 columns]


# Checagem

In [276]:
table_ice.info()
table_temp.info()
table_ocean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 2 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   year                            46 non-null     int64  
 1   media anual - extensão do gelo  46 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 868.0 bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           145 non-null    int64  
 1   media_station  145 non-null    float64
 2   media_land     145 non-null    float64
dtypes: float64(2), int64(1)
memory usage: 3.5 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    174 non-null    int64  
 1   resul

In [275]:
table_ocean

,year,result
0,1851,-0.05
1,1852,0.01
2,1853,0.06
3,1854,0.00
4,1855,0.00
...,...,...
169,2020,0.75
170,2021,0.70
171,2022,0.64
172,2023,0.67


In [277]:
table_ice

,year,media anual - extensão do gelo
0,1979,24.005833
1,1980,23.554167
2,1981,23.533333
3,1982,24.062500
4,1983,23.697500
5,1984,23.348333
6,1985,23.585000
7,1986,23.268333
8,1987,23.663636
9,1988,24.031818


In [278]:
table_temp

,year,media_station,media_land
0,1880,-0.356667,-0.180000
1,1881,-0.289167,-0.092500
2,1882,-0.393333,-0.113333
3,1883,-0.377500,-0.175833
4,1884,-0.508333,-0.285000
...,...,...,...
140,2020,1.315833,1.007500
141,2021,1.143333,0.849167
142,2022,1.163333,0.894167
143,2023,1.441667,1.173333


# Salvando em banco de dados


In [279]:
import sqlite3
conn = sqlite3.connect('bd1.db')
table_ocean.to_sql('table_ocean', conn, if_exists='replace', index=False)


174

In [280]:
query = "SELECT * FROM table_ocean"
df = pd.read_sql(query, conn)
print(df)
conn.close()

     year  result
0    1851   -0.05
1    1852    0.01
2    1853    0.06
3    1854    0.00
4    1855    0.00
..    ...     ...
169  2020    0.75
170  2021    0.70
171  2022    0.64
172  2023    0.67
173  2024    0.94

[174 rows x 2 columns]


In [281]:
conn = sqlite3.connect('bd1.db')
table_ice.to_sql('table_ice', conn, if_exists='replace', index=False)


46

In [282]:
query = "SELECT * FROM table_ice"
df = pd.read_sql(query, conn)
print(df)
conn.close()

    year  media anual - extensão do gelo
0   1979                       24.005833
1   1980                       23.554167
2   1981                       23.533333
3   1982                       24.062500
4   1983                       23.697500
5   1984                       23.348333
6   1985                       23.585000
7   1986                       23.268333
8   1987                       23.663636
9   1988                       24.031818
10  1989                       23.367500
11  1990                       23.082500
12  1991                       23.268333
13  1992                       23.495833
14  1993                       23.316667
15  1994                       23.760000
16  1995                       23.186667
17  1996                       23.591667
18  1997                       23.032500
19  1998                       23.470833
20  1999                       23.425833
21  2000                       23.243333
22  2001                       23.250000
23  2002        

In [283]:
conn = sqlite3.connect('bd1.db')
table_temp.to_sql('table_temp', conn, if_exists='replace', index=False)

145

In [284]:
query = "SELECT * FROM table_temp"
df = pd.read_sql(query, conn)
print(df)
conn.close()

     year  media_station  media_land
0    1880      -0.356667   -0.180000
1    1881      -0.289167   -0.092500
2    1882      -0.393333   -0.113333
3    1883      -0.377500   -0.175833
4    1884      -0.508333   -0.285000
..    ...            ...         ...
140  2020       1.315833    1.007500
141  2021       1.143333    0.849167
142  2022       1.163333    0.894167
143  2023       1.441667    1.173333
144  2024       1.542000    1.285000

[145 rows x 3 columns]
